Define the growth function which takes in the current state of the system y, the parameters p, and the presence or absence of inhibitors inh as arguments. This function describes the rates of change of nutrients n, the degrading enzyme b, and antibiotic a levels over time t under specific conditions. 

The simulate function is used to solve the defined ODEs for a given initial condition n0 and a0 over a certain time span using the solve_ivp function from the SciPy library. This function returns the simulated levels of nutrients, the degrading enzyme, and antibiotic over time.

The objective function computes the sum of squared differences between the simulated nutrient levels and the observed experimental nutrient levels for all time points and initial conditions. This function is typically used in parameter estimation where the goal is to find the parameters p that minimize the difference between the simulated and experimental results.

Here, the growth function is defined to be flexible enough to capture the general trend of experimental growth curves. Many of these exhibit approximately 2-stage growth. A simple logistic model or Monod equation does not capture the growth curve well. The term 'g' is re-defined to approximate this behavior (to some extent). There might be better alternatives.

In the following code, the initial parameter values are from the NN estimated. Let's see if Sciy optimize can generate better fits for some or all isolates.

In [1]:
import numpy as np
from scipy.integrate import odeint, solve_ivp
from scipy.optimize import minimize

%run ../simulation/growth_complete.ipynb   #defines growth dynamics and simulation method

In [4]:

#
# 1st round of parameter estimation.
#
# Bounds for the parameter values
# alpha, Ks, theta, Ln, kappab, phimax, gamma, betamin, db, c
parameter_names = ['alpha','Ks','theta', 'Ln', 'kappab', 'phimax', 'gamma', 'betamin', 'db', 'c']

#old_estimate = np.load("../estimated/v7_NN_round1.npy")
#old_estimate = np.mean(old_estimate, axis = 1)

estimated_gr= np.load('../estimated/gravg2.npy')  #estimated alpha, Ks, and theta. will be fixed in subsequent optimization.

# print(old_estimate.shape)
# p0 = np.array([(lower + upper) / 2 for lower, upper in parameter_bounds])

# load the experimental data
experimental_all = np.load('../isolates/avg_OD_all_isolates.npy')
strain_ids = np.load('../isolates/strain_ids.npy', allow_pickle=True)

print("experimental_all shape:", experimental_all.shape)

num_parameter_sets = 311 # Run the optimization for each strain, using averaged growth curves.
num_runs = 10

for k in range(num_runs):
    optimal_parameters = []
    initial_guesses = []
    for i in range(num_parameter_sets):
        experimental = experimental_all[i]
        
        # Define the constraint function for parameter bounds, which are required by other optimization methods.
        #constraint = {'type': 'ineq', 'fun': lambda p: np.array([p[j] - parameter_bounds[j][0] for j in range(len(p))])}
        
        #p0 = old_estimate[i]
        p0 = np.zeros(10)
        p0[0:3] = estimated_gr[i, 0:3]

        #set the bounds such that the first 3 parameters will not change during optimization.
        parameter_bounds = [(p0[0], p0[0]), (p0[1], p0[1]), (p0[2], p0[2]), 
                            (0, 0.8), (0, 6), (0, 6),
                            (0.0, 6), (0, 1), (0, 6), (0, 1)]

        #set the means and standard deviations of the normal distribution
        parameter_means = [p0[0], p0[1], p0[2], 0.4, 3, 3, 1.35, 0.5, 3, 0.146]
        parameter_sds = [0, 0, 0, 0.4/3, 1, 1, 0.55/3, 0.5/3, 1, 0.0487]

        for j in range(3, 10):
            p0[j] = np.random.normal(parameter_means[j], parameter_sds[j])
        #print(p0)  
        initial_guesses.append(p0)
        
        result = minimize(objective, p0, args=(experimental,), method='Nelder-Mead', tol = 1e-7, bounds=parameter_bounds)
        optimal_parameters.append(result.x)
        interval = 1 if i<10 else 10
        if ((i+1) % interval == 0):
            print(strain_ids[i])
    #        for p in optimal_parameters[i]:
            for j in range(len(optimal_parameters[i])):
                name = parameter_names[j]
                p = optimal_parameters[i][j]
                print(name, f"={p:.2f}", end="; ")
            print()  # Print a new line after each simulation

    # Convert the list of optimized parameters to a numpy array
    initial_guess_array = np.array(initial_guesses)
    optimal_parameters_array = np.array(optimal_parameters)
    # Save the optimized parameters to a .npy file
    np.save(('../estimated/scipy_all_para_v14_' + str(k)), optimal_parameters_array)
    np.save(('../estimated/scipy_initials_v14_' + str(k)), initial_guess_array)


experimental_all shape: (311, 3, 145)
2004
alpha =0.87; Ks =0.11; theta =1.83; Ln =0.06; kappab =0.06; phimax =0.02; gamma =2.68; betamin =0.30; db =6.00; c =0.00; 
2005
alpha =1.03; Ks =0.15; theta =2.21; Ln =0.07; kappab =0.00; phimax =0.00; gamma =1.03; betamin =0.98; db =4.35; c =0.69; 
2007
alpha =0.88; Ks =0.17; theta =2.63; Ln =0.43; kappab =5.95; phimax =2.55; gamma =1.32; betamin =0.77; db =6.00; c =0.15; 
2009
alpha =0.84; Ks =0.19; theta =3.64; Ln =0.26; kappab =0.00; phimax =3.54; gamma =0.88; betamin =0.00; db =5.69; c =1.00; 
2039
alpha =0.84; Ks =0.19; theta =3.38; Ln =0.18; kappab =0.09; phimax =1.71; gamma =1.66; betamin =0.49; db =2.39; c =0.05; 
2045
alpha =0.85; Ks =0.20; theta =3.85; Ln =0.54; kappab =4.01; phimax =3.54; gamma =2.11; betamin =0.55; db =5.34; c =0.05; 
2050
alpha =0.78; Ks =0.18; theta =3.35; Ln =0.39; kappab =3.41; phimax =3.73; gamma =4.33; betamin =0.00; db =1.11; c =0.25; 


/tmp/ipykernel_2866791/751365360.py:52: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(objective, p0, args=(experimental,), method='Nelder-Mead', tol = 1e-7, bounds=parameter_bounds)


2055
alpha =0.91; Ks =0.19; theta =3.03; Ln =0.32; kappab =6.00; phimax =6.00; gamma =1.00; betamin =0.00; db =6.00; c =1.00; 
2060
alpha =1.23; Ks =0.10; theta =1.71; Ln =0.15; kappab =0.11; phimax =0.06; gamma =1.19; betamin =0.99; db =6.00; c =0.08; 
2091
alpha =0.88; Ks =0.19; theta =3.40; Ln =0.35; kappab =6.00; phimax =0.00; gamma =2.91; betamin =0.00; db =6.00; c =0.30; 
2175
alpha =0.79; Ks =0.17; theta =3.25; Ln =0.30; kappab =0.00; phimax =5.86; gamma =0.95; betamin =0.00; db =0.00; c =0.88; 
2289
alpha =0.84; Ks =0.19; theta =3.58; Ln =0.38; kappab =5.09; phimax =0.00; gamma =2.56; betamin =0.00; db =5.63; c =0.33; 
2345
alpha =0.69; Ks =0.20; theta =4.24; Ln =0.15; kappab =0.08; phimax =1.47; gamma =0.69; betamin =1.00; db =1.20; c =0.54; 
2405
alpha =0.64; Ks =0.20; theta =4.62; Ln =0.21; kappab =1.45; phimax =2.63; gamma =2.28; betamin =0.43; db =2.92; c =0.24; 
2449
alpha =0.95; Ks =0.13; theta =2.03; Ln =0.31; kappab =0.70; phimax =0.00; gamma =2.64; betamin =0.46; db =

The following code checks the quality of estimated parameters
1. Load the estimated parameters 
2. For each parameter set, conduct simulations with the defined timespan for the three conditions: (0, 0), (10, 0), (3, 10)
3. In each simulation, n0 is set by using the first value of the corresponding experimental data
4. For each parameter set, generate a 3-panel plot. Each panel shows the overlay of simulated data and experimental data for one of the 3 conditions.
5. When doing the optimization, the experimental time points are not considered. Instead, a reasonable time span is used to simulate time courses. It is close but not identical to experimental time span. This can be corrected later by re-scaling the time axis of the simulation. 